<a href="https://colab.research.google.com/github/CSpanias/spam_filter_bootcamp/blob/Charalampos/spam_filter_GradientBoosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import string
import matplotlib.pyplot as plt
import nltk
import seaborn as sns

from nltk.corpus import stopwords
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV

from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn.metrics import mean_absolute_error

In [4]:
# Load data
from google.colab import files
uploaded = files.upload()

Saving emails.csv to emails.csv


In [5]:
# read the CSV file
df = pd.read_csv('emails.csv')

# print the first 5 rows of data
df.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [6]:
# print the number of rows and columns
df.shape
# check for duplicates
print(df.duplicated().sum())
# remove duplicates
df.drop_duplicates(inplace=True)
# check for missing values
df.isna().sum()

33


text    0
spam    0
dtype: int64

In [7]:
import nltk
# download the stopwords package
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
def process_text(text):
	"""
		Returns a list of strings (individual words) of a text (paragraph) excluding
		punctuation, stopwords.

			Parameters
			----------
				text: An object (string) variable.

			Returns
			-------
				clean_words: A list of objects (string).
		"""
	# create an empty list to later store the rows (emails) without punctuation
	no_punctuation = []
	# create an empty list to later store the words of each row (email)
	clean_words = []
	# for every row (email) in the text column
	for char in text:
		# remove every punctuation symbol
		if char not in string.punctuation:
			# append the free-of-punctuation row in the list "no_punctuation"
			no_punctuation.append(char)
	# insert an empty space between words
	no_punctuation = ''.join(no_punctuation)

	# for every row (email without punctuation symbols) in
	# the "no_punctuation" list split the text into words
	for word in no_punctuation.split():
		# convert every word in lowercase
		word = word.lower()
		# if this word is not a stopword
		if word not in stopwords.words('english'):
			# append the free-of-stopwords words in the list "clean_words"
			clean_words.append(word)
	# give back the list "clean_words"
	return clean_words

In [9]:
df['text'].head().apply(process_text)

0    [subject, naturally, irresistible, corporate, ...
1    [subject, stock, trading, gunslinger, fanny, m...
2    [subject, unbelievable, new, homes, made, easy...
3    [subject, 4, color, printing, special, request...
4    [subject, money, get, software, cds, software,...
Name: text, dtype: object

In [10]:
# convert a collection of text to a matrix of tokens (*bow = bog of words)
messages_bow = CountVectorizer(analyzer=process_text).fit_transform(df['text'])

In [11]:
# assign X and y variables
X = messages_bow
y = df['spam']

# split training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,
                                                    random_state=1)

In [14]:
# instantiate classifier with default parameters
model = ensemble.GradientBoostingClassifier()
    
# declare parameters for hyperparameter tuning
params = [
          {'n_estimators': [10, 100, 150], 'learning_rate':[0.1, 0.3]}
          ]

# create the grid search object
gs = GridSearchCV(model, params, scoring='f1', cv=5, verbose=3)

# fit the gs object
gs.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END learning_rate=0.1, n_estimators=10;, score=0.484 total time=   1.4s
[CV 2/5] END learning_rate=0.1, n_estimators=10;, score=0.851 total time=   1.4s
[CV 3/5] END learning_rate=0.1, n_estimators=10;, score=0.623 total time=   1.4s
[CV 4/5] END learning_rate=0.1, n_estimators=10;, score=0.865 total time=   1.5s
[CV 5/5] END learning_rate=0.1, n_estimators=10;, score=0.858 total time=   1.5s
[CV 1/5] END learning_rate=0.1, n_estimators=100;, score=0.956 total time=  13.3s
[CV 2/5] END learning_rate=0.1, n_estimators=100;, score=0.938 total time=  13.4s
[CV 3/5] END learning_rate=0.1, n_estimators=100;, score=0.923 total time=  13.7s
[CV 4/5] END learning_rate=0.1, n_estimators=100;, score=0.946 total time=  13.7s
[CV 5/5] END learning_rate=0.1, n_estimators=100;, score=0.923 total time=  13.8s
[CV 1/5] END learning_rate=0.1, n_estimators=150;, score=0.956 total time=  19.5s
[CV 2/5] END learning_rate=0.1, n_estimator

GridSearchCV(cv=5, estimator=GradientBoostingClassifier(),
             param_grid=[{'learning_rate': [0.1, 0.3],
                          'n_estimators': [10, 100, 150]}],
             scoring='f1', verbose=3)

In [18]:
# get the best parameters of the model
print('Parameters that give the best results:', gs.best_params_)

# get the model with the highest training accuracy
print('\nThe model with the best training accucary has a score of:', (round(gs.best_score_, 4)))

# print estimator that was chosen by the GridSearch
print('\nEstimator that was chosen by the search:', gs.best_estimator_)

# calculate GridSearch CV score on test set
print('\nGridSearch CV score on test set:', round(gs.score(X_test, y_test), 4))

Parameters that give the best results: {'learning_rate': 0.3, 'n_estimators': 100}

The model with the best training accucary has a score of: 0.9467

Estimator that was chosen by the search: GradientBoostingClassifier(learning_rate=0.3)

GridSearch CV score on test set: 0.9456
